In [4]:
import re
import pandas as pd
import numpy as np
data_cleaned=pd.read_csv("../data_12.csv")
data_cleaned["ROLL_NO"]=data_cleaned[data_cleaned.columns[6]]
data_cleaned.drop(data_cleaned.columns[6],1,inplace=True)
data_cleaned[data_cleaned.columns[0]]=data_cleaned[data_cleaned.columns[0]].apply(lambda x: re.sub('(....$)',"",x))
data_cleaned.drop(" INSTRUCTOR NAME",1,inplace=True)
data_cleaned=data_cleaned[data_cleaned[" SEMESTER"]!=" SEMESTER"]
data_cleaned[data_cleaned.columns[1]]=pd.to_numeric(data_cleaned.loc[:,data_cleaned.columns[1]])
data_cleaned[data_cleaned.columns[0]]=pd.to_numeric(data_cleaned.loc[:,data_cleaned.columns[0]])
data_cleaned.ROLL_NO=data_cleaned.ROLL_NO.astype(str)
roll_nos=data_cleaned.ROLL_NO.values
corpus=data_cleaned[" COURSE No. "].values

from sklearn.feature_extraction.text import CountVectorizer
vectorizerX = CountVectorizer(min_df=1)
vectorizerY = CountVectorizer(min_df=1)
X = vectorizerX.fit_transform(corpus)
Y = vectorizerY.fit_transform(roll_nos)

len_courses=len(vectorizerX.get_feature_names())
len_students=len(vectorizerY.get_feature_names())-1


data_cleaned.ROLL_NO=data_cleaned.ROLL_NO.astype(float)
rolled=(vectorizerY.get_feature_names())
for i,roll in enumerate(rolled):
    rolled[i]=float(roll)
rolled=rolled[:-1]    


dat_mat_3=np.zeros((len_students,len_courses))
dat_mat_4=np.zeros((len_students,len_courses))
dat_mat_5=np.zeros((len_students,len_courses))
dat_mat_6=np.zeros((len_students,len_courses))
dat_mat_7=np.zeros((len_students,len_courses))
dat_mat_8=np.zeros((len_students,len_courses))

for i,roll in enumerate(rolled):
    first_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2012)&(data_cleaned[data_cleaned.columns[1]]==1)
    second_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2012)&(data_cleaned[data_cleaned.columns[1]]==2)
    summer_1=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2012)&(data_cleaned[data_cleaned.columns[1]]==3)
    third_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2013)&(data_cleaned[data_cleaned.columns[1]]==1)
    fourth_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2013)&(data_cleaned[data_cleaned.columns[1]]==2)
    summer_2=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2013)&(data_cleaned[data_cleaned.columns[1]]==3)
    fifth_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2014)&(data_cleaned[data_cleaned.columns[1]]==1)
    sixth_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2014)&(data_cleaned[data_cleaned.columns[1]]==2)
    summer_3=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2014)&(data_cleaned[data_cleaned.columns[1]]==3)
    seventh_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2015)&(data_cleaned[data_cleaned.columns[1]]==1)
    eigth_sem=(data_cleaned[data_cleaned.columns[5]]==roll)&(data_cleaned[data_cleaned.columns[0]]==2015)&(data_cleaned[data_cleaned.columns[1]]==2)
    courses_3=third_sem|first_sem|second_sem|summer_1
    courses_4=third_sem|first_sem|second_sem|fourth_sem|summer_1
    courses_5=fifth_sem|fourth_sem|third_sem|first_sem|second_sem|summer_2|summer_1
    courses_6=sixth_sem|fifth_sem|fourth_sem|third_sem|first_sem|second_sem|summer_2|summer_1    
    courses_7=seventh_sem|summer_3|sixth_sem|fifth_sem|fourth_sem|third_sem|first_sem|second_sem|summer_2|summer_1    
    courses_8=eigth_sem|seventh_sem|summer_3|sixth_sem|fifth_sem|fourth_sem|third_sem|first_sem|second_sem|summer_2|summer_1    

    dat_mat_3[i]=sum(vectorizerX.transform(data_cleaned[courses_3][" COURSE No. "].values).toarray())
    dat_mat_4[i]=sum(vectorizerX.transform(data_cleaned[courses_4][" COURSE No. "].values).toarray())
    dat_mat_5[i]=sum(vectorizerX.transform(data_cleaned[courses_5][" COURSE No. "].values).toarray())
    dat_mat_6[i]=sum(vectorizerX.transform(data_cleaned[courses_6][" COURSE No. "].values).toarray())
    dat_mat_7[i]=sum(vectorizerX.transform(data_cleaned[courses_7][" COURSE No. "].values).toarray())
    dat_mat_8[i]=sum(vectorizerX.transform(data_cleaned[courses_8][" COURSE No. "].values).toarray())

from sklearn.neighbors import NearestNeighbors
def find_neighbr(querry,roll_no):
    if (roll_no>14000&roll_no<15000):
        nbrs = NearestNeighbors(n_neighbors=50, algorithm='auto').fit(dat_mat_5)
        distances, indices = nbrs.kneighbors(querry)
        return indices 
    elif (roll_no>150000&roll_no<151000):
        nbrs = NearestNeighbors(n_neighbors=50, algorithm='auto').fit(dat_mat_3)
        distances, indices = nbrs.kneighbors(querry)
        return indices 
    elif (roll_no>13000&roll_no<14000):
        nbrs = NearestNeighbors(n_neighbors=15, algorithm='auto').fit(dat_mat_7)
        distances, indices = nbrs.kneighbors(querry)
        return indices 

def predict_courses(querry,roll_no):
    print(roll_no)
    if ((roll_no>14000)&(roll_no<15000)):
        indice=find_neighbr(querry,roll_no)
        course_vec=dat_mat_6[indice[0,:],:]-dat_mat_5[indice[0,:],:]
        print("indice1")
        return course_vec
    elif ((roll_no>150000)&(roll_no<151000)):
        indice=find_neighbr(querry,roll_no)
        course_vec=dat_mat_4[indice[0,:],:]-dat_mat_3[indice[0,:],:]
        print("indice2")
        return course_vec
    elif ((roll_no>13000)&(roll_no<14000)):
        indice=find_neighbr(querry,roll_no)
        print("indice3")
        course_vec=dat_mat_8[indice[0,:],:]-dat_mat_7[indice[0,:],:]
        return course_vec
querry=dat_mat_7[311,:]
roll_no=14250
#vectorizerX.inverse_transform(predict_courses(querry,roll_no))
vectorizerX.inverse_transform(sum(predict_courses(querry,roll_no)))
#predict_courses(querry,roll_no).shape

14250
indice1


/home/abhinav/miniconda3/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[array(['art401a', 'art406a', 'chm645a', 'eco221a', 'eco501a', 'ee210a',
        'ee250a', 'ee301a', 'ee311a', 'ee321a', 'ee340a', 'ee360a',
        'ee381a', 'ee392a', 'ee609a', 'ee627a', 'eng433a', 'eng436a',
        'eng439a', 'ime642a', 'mba635a', 'me774a', 'mso201a', 'mth552a',
        'nt614a', 'phi141a', 'psy454a', 'psy458a', 'soc171a', 'soc479a',
        'soc481a'], 
       dtype='<U7')]